In [ ]:
async def finance_agent_generator(query: str):
    graph = create_finance_agent_graph()
    initial_state = AgentState(
        query=query,
        processed_query="",
        use_graph_db=False,
        graph_db_result=[],
        expanded_queries=[],
        subgraphs_to_execute=3,
        subgraph_results={},
        final_response=""
    )
    current_state = initial_state
    
    last_node = None
    async for chunk in graph.astream(current_state, stream_mode="updates"):
        for node, node_state in chunk.items():
            if isinstance(node_state, dict):
                yield f"data: {json.dumps({'type': 'step', 'content': node})}\n\n"
                last_node = node
            else:
                print(f"Unexpected state type for node {node}: {type(node_state)}.")
        
        # After processing all nodes, check if we have a final response
        if last_node and 'final_response' in chunk[last_node]:
            final_response = chunk[last_node]['final_response']
            if final_response:
                yield f"data: {json.dumps({'type': 'final', 'content': final_response})}\n\n"

    yield "data: [DONE]\n\n"